In [1]:
import os
from shutil import copyfile


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from PIL import Image
import pickle

from matplotlib import image
from tqdm import tqdm

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [2]:
DATA_PATH = os.getcwd() + '/Data/'

---

# Rename and Save

In [3]:
ids = next(os.walk(f"{DATA_PATH}predicted_corrected/"))[2]
print("No. of images = ", len(ids))

No. of images =  25


In [4]:
# Finding out how much we can crop
img = 255 - cv.imread(f"{DATA_PATH}predicted_corrected/{ids[0]}",0)
edged = cv.Canny(img, 100, 255)
thresh = cv.adaptiveThreshold(edged, 255, 1, 1, 11, 2)
thresh = cv.dilate(thresh, None, iterations=15)
thresh = cv.erode(thresh, None, iterations=15)
contours, hierarchy = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)
armax = 0
X = 0
Y = 0
W = 0
H = 0
for cnt in contours:
    x, y, w, h = cv.boundingRect(cnt)
    if w*h > armax:
        armax = w*h
        X = x-200
        Y = y-200
        W = w+400
        H = h+400

#Getting the length of the digits
max_len = 0
for i in ids:
    ii = int(i.replace("F_","").replace(".png",""))*5
    if len(str(ii)) > max_len:
        max_len = len(str(ii))

# Saving the Images in the Apropriate folder, with the right name
for i in tqdm(ids):
    ii = int(i.replace("F_","").replace(".png",""))*5
    k = max_len - len(str(ii))
    ii = k*"0" + str(ii)
    new_name = i.replace(i.replace("F_","").replace(".png",""),ii).replace(".png",".tif")
    new_name_png = i.replace(i.replace("F_","").replace(".png",""),ii)
    new_name_bmp = i.replace(i.replace("F_","").replace(".png",""),ii).replace(".png",".bmp")
    img = 255 - cv.imread(f"{DATA_PATH}predicted_corrected/{i}",0)
    img_color = cv.imread(f"{DATA_PATH}predicted_corrected/raw_color/{i}")
    img_roi = cv.imread(f"{DATA_PATH}predicted_corrected/roi/{i}",0)
    _,img_roi = cv.threshold(img_roi, 128, 255, cv.THRESH_BINARY)
    img_color = cv.bitwise_and(img_color,img_color,mask = img.astype(np.uint8))
    cv.imwrite(f"{DATA_PATH}predicted_completed/{new_name}",img[Y:(Y+H),X:(X+W)])
    cv.imwrite(f"{DATA_PATH}predicted_completed/png/{new_name_png}",img[Y:(Y+H),X:(X+W)])
    cv.imwrite(f"{DATA_PATH}predicted_completed/png_color/{new_name_png}",img_color[Y:(Y+H),X:(X+W)])
    cv.imwrite(f"{DATA_PATH}predicted_completed/tif_color/{new_name}",img_color[Y:(Y+H),X:(X+W)])
    cv.imwrite(f"{DATA_PATH}predicted_completed/roi/{new_name_bmp}",img_roi[Y:(Y+H),X:(X+W)])

100%|███████████████████████████████████████████████████████████████████████████████████| 25/25 [01:14<00:00,  2.99s/it]


In [38]:
# Finding out how much we can crop
'''img = 255 - cv.imread(f"{DATA_PATH}predicted_corrected/{ids[0]}",0)
edged = cv.Canny(img, 100, 255)
thresh = cv.adaptiveThreshold(edged, 255, 1, 1, 11, 2)
thresh = cv.dilate(thresh, None, iterations=15)
thresh = cv.erode(thresh, None, iterations=15)
contours, hierarchy = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)
armax = 0
X = 0
Y = 0
W = 0
H = 0
for cnt in contours:
    x, y, w, h = cv.boundingRect(cnt)
    if w*h > armax:
        armax = w*h
        X = x-200
        Y = y-200
        W = w+400
        H = h+400

#Getting the length of the digits
max_len = 0
for i in ids:
    ii = int(i.replace("F_","").replace(".png",""))*5
    if len(str(ii)) > max_len:
        max_len = len(str(ii))

# Saving the Images in the Apropriate folder, with the right name
for i in ids:
    ii = int(i.replace("F_","").replace(".png",""))*5
    k = max_len - len(str(ii))
    ii = k*"0" + str(ii)
    new_name = i.replace(i.replace("F_","").replace(".png",""),ii).replace(".png",".tif")
    new_name_png = i.replace(i.replace("F_","").replace(".png",""),ii)
    new_name_bmp = i.replace(i.replace("F_","").replace(".png",""),ii).replace(".png",".bmp")
    img = 255 - cv.imread(f"{DATA_PATH}predicted_corrected/{i}",0)
    img_color = cv.imread(f"{DATA_PATH}predicted_corrected/raw_color/{i}")
    img_roi = img.copy()
    edged = cv.Canny(img_roi, 100, 255)
    thresh = cv.adaptiveThreshold(edged, 255, 1, 1, 11, 2)
    thresh = cv.dilate(thresh, None, iterations=200)
    thresh = cv.erode(thresh, None, iterations=180)
    contours, hierarchy = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)
    armax = 0
    for cnt in contours:
        x, y, w, h = cv.boundingRect(cnt)
        if w*h > armax:
            cnt_max = cnt
            armax = w*h
    img_roi = cv.fillPoly(np.zeros(np.shape(img_roi)), pts =[cnt_max], color=(255,255,255))
    img_color = cv.bitwise_and(img_color,img_color,mask = img.astype(np.uint8))
    cv.imwrite(f"{DATA_PATH}predicted_completed/{new_name}",img[Y:(Y+H),X:(X+W)])
    cv.imwrite(f"{DATA_PATH}predicted_completed/png/{new_name_png}",img[Y:(Y+H),X:(X+W)])
    cv.imwrite(f"{DATA_PATH}predicted_completed/png_color/{new_name_png}",img_color[Y:(Y+H),X:(X+W)])
    cv.imwrite(f"{DATA_PATH}predicted_completed/tif_color/{new_name}",img_color[Y:(Y+H),X:(X+W)])
    cv.imwrite(f"{DATA_PATH}predicted_completed/roi/{new_name_bmp}",img_roi[Y:(Y+H),X:(X+W)])'''

# Complete

In [5]:
ids = next(os.walk(f"{DATA_PATH}predicted_completed/"))[2]
print("No. of images = ", len(ids))

No. of images =  191


In [7]:
copy_list = []
for i in range(len(ids)-1):
    ii = int(ids[i].replace("F_","").replace(".tif",""))
    iii = int(ids[i+1].replace("F_","").replace(".tif",""))
    if not iii == ii + 1:
        num_missing = iii-ii-1
        for j in range(1,num_missing+1):
            new_id = ii + j
            base_name = ids[i]
            name = ids[i].replace(str(ii),"").replace(".tif","")
            new_name = name + str(new_id) + ".tif"
            copy_dict = {"id":new_id,"base_name":base_name,"new_name":new_name}
            copy_list.append(copy_dict)

for i in range(len(copy_list)):
    src = copy_list[i]["base_name"]
    dst = copy_list[i]["new_name"]
    copyfile(f"{DATA_PATH}predicted_completed/{src}",f"{DATA_PATH}predicted_completed/{dst}")

In [6]:
ids = next(os.walk(f"{DATA_PATH}predicted_completed/tif_color/"))[2]
print("No. of images = ", len(ids))

No. of images =  25


In [7]:
copy_list = []
for i in range(len(ids)-1):
    ii = int(ids[i].replace("F_","").replace(".tif",""))
    iii = int(ids[i+1].replace("F_","").replace(".tif",""))
    if not iii == ii + 1:
        num_missing = iii-ii-1
        for j in range(1,num_missing+1):
            new_id = ii + j
            base_name = ids[i]
            name = ids[i].replace(str(ii),"").replace(".tif","")
            new_name = name + str(new_id) + ".tif"
            copy_dict = {"id":new_id,"base_name":base_name,"new_name":new_name}
            copy_list.append(copy_dict)

for i in range(len(copy_list)):
    src = copy_list[i]["base_name"]
    dst = copy_list[i]["new_name"]
    copyfile(f"{DATA_PATH}predicted_completed/tif_color/{src}",f"{DATA_PATH}predicted_completed/tif_color/{dst}")

# Downscale for reasonable runtime

In [38]:
ids = next(os.walk(f"{DATA_PATH}predicted_completed/"))[2]
print("No. of images = ", len(ids))

No. of images =  191


In [39]:
for i in ids:
    img = cv.imread(f"{DATA_PATH}predicted_completed/{i}",0)
    scale_percent = 33 # percent of original size
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    img_scaled = cv.resize(img, dim, interpolation = cv.INTER_AREA)
    _,thresh = cv.threshold(img_scaled, 127, 255, cv.THRESH_BINARY)
    _,thresh2 = cv.threshold(255-img_scaled, 254, 255, cv.THRESH_BINARY)
    _,thresh3 = cv.threshold(255-thresh2-thresh, 127, 127, cv.THRESH_BINARY)
    img_scaled = thresh3 + thresh
    cv.imwrite(f"{DATA_PATH}predicted_completed/downscaled/{i}",img_scaled)